In [2]:
import pandas as pd
import numpy as np
import glob
from scipy import stats
from scipy.stats import kurtosis, skew

In [3]:
def spherical_dist_populate(lat_lis, lon_lis, r=3958.75):
    lat_mtx = np.array([lat_lis]).T * np.pi / 180
    lon_mtx = np.array([lon_lis]).T * np.pi / 180

    cos_lat_i = np.cos(lat_mtx)
    cos_lat_j = np.cos(lat_mtx)
    cos_lat_J = np.repeat(cos_lat_j, len(lat_mtx), axis=1).T

    lat_Mtx = np.repeat(lat_mtx, len(lat_mtx), axis=1).T
    cos_lat_d = np.cos(lat_mtx - lat_Mtx)

    lon_Mtx = np.repeat(lon_mtx, len(lon_mtx), axis=1).T
    cos_lon_d = np.cos(lon_mtx - lon_Mtx)

    mtx = r * np.arccos(cos_lat_d - cos_lat_i*cos_lat_J*(1 - cos_lon_d))
    return mtx

In [17]:
# Get list of files in folder
files = []
for file in glob.glob("/home/server/pi/homes/woodilla/Data/GFW_point/Patagonia_Shelf/feather" + "/*.feather"):
    files.append(file)
nfiles = sorted(files)

1096

In [25]:
for i in nfiles[0:2]:
    indat = pd.read_feather(i)
    indat = indat.sort_values('mmsi')
    outdat = indat
    outdat['lat_avg'] = outdat.groupby('mmsi').lat.transform('mean')
    outdat['lon_avg'] = outdat.groupby('mmsi').lon.transform('mean')
    outdat = outdat.groupby('mmsi').first().reset_index()

    outdat = outdat[['timestamp', 'year', 'month', 'day', 'mmsi', 'lat', 'lon', \
                    'segment_id', 'message_id', 'type', 'speed', 'course', 'heading', 'shipname', 'callsign', \
                     'destination', 'elevation_m', 'distance_from_shore_m', 'distance_from_port_m', 'nnet_score', \
                     'logistic_score', 'flag', 'geartype', 'length', 'tonnage', 'engine_power', 'active_2012', \
                     'active_2013', 'active_2014', 'active_2015', 'active_2016']]
    date = f"{outdat['year'][1]}-" + f"{outdat['month'][1]}".zfill(2) + f"-" + f"{outdat['day'][1]}".zfill(2)

    matdat = pd.DataFrame(spherical_dist_populate(outdat['lat'], outdat['lon']))
    matdat = matdat.rename(index=outdat.mmsi, columns = outdat.mmsi)    
    tmatdat = matdat.where(np.triu(np.ones(matdat.shape)).astype(np.bool))
    tmatdat = tmatdat.stack().reset_index()
    tmatdat['date'] = date
    tmatdat.columns = ['vessel_A', 'vessel_B', 'distance', 'date']
    tmatdat = tmatdat[['date', 'vessel_A', 'vessel_B', 'distance']]
    tmatdat.to_feather('/home/server/pi/homes/woodilla/Data/GFW_point/Patagonia_Shelf/distance_data/circle_measure/' + date + '_dmatrix' + '.feather')

    

In [ ]:
#Distribution variables
newdat = pd.DataFrame({'date': date, 'mean': np.mean(tmatdat['distance']), 'std': np.std(tmatdat['distance']), \
                       'kurtosis': kurtosis(tmatdat['distance']), 'skewness': skew(tmatdat['distance'])}, index=[0])
dist_dat = dist_dat.append(newdat, ignore_index=True)
dist_dat.to_feather('~/Projects/Patagonia-EDA/distance_sample.feather')
dist_dat = pd.DataFrame(columns = ['date', 'mean', 'std', 'kurtosis'])

In [26]:
tdat = pd.read_feather('/home/server/pi/homes/woodilla/Data/GFW_point/Patagonia_Shelf/distance_data/circle_measure/2016-01-03_dmatrix.feather')
tdat

date    vessel_A    vessel_B     distance
0        2016-01-03           9           9     0.000000
1        2016-01-03           9          11   643.876044
2        2016-01-03           9         372   142.189277
3        2016-01-03           9        1000   576.029560
4        2016-01-03           9        1942   520.659106
5        2016-01-03           9        1957   651.036057
6        2016-01-03           9        1958   651.033563
7        2016-01-03           9        2409    21.968897
8        2016-01-03           9        2411   143.695975
9        2016-01-03           9        2829    22.233938
10       2016-01-03           9        2902   143.723193
11       2016-01-03           9        2940    22.501268
12       2016-01-03           9        3098   651.318750
13       2016-01-03           9        3099   651.321419
14       2016-01-03           9        3340    78.973455
15       2016-01-03           9        3496     9.060434
16       2016-01-03           9        3604   154.496479
17       2016-01-03           9        3887     6.882257
18       2016-01-03           9        5625   557.424669
19       2016-01-03           9       36407   643.884430
20       2016-01-03           9      905000     5.214809
21       2016-01-03           9     1193046   557.730161
22       2016-01-03           9     1193047   562.736149
23       2016-01-03           9     1320868  1997.105957
24       2016-01-03           9     4444444     2.196026
25       2016-01-03           9     5300142    21.852772
26       2016-01-03           9     7010001   625.884806
27       2016-01-03           9     7010003   863.472158
28       2016-01-03           9     7010004   539.771641
29       2016-01-03           9     7010005   960.508833
...             ...         ...         ...          ...
2811976  2016-01-03   997705001   999999999   457.945980
2811977  2016-01-03   997705001  1040354684  1330.580080
2811978  2016-01-03   997753940   997753940     0.000000
2811979  2016-01-03   997753940   998666182   863.487632
2811980  2016-01-03   997753940   998666200   916.398847
2811981  2016-01-03   997753940   999457666     1.883550
2811982  2016-01-03   997753940   999999989  1417.172663
2811983  2016-01-03   997753940   999999999   977.366265
2811984  2016-01-03   997753940  1040354684   844.052710
2811985  2016-01-03   998666182   998666182     0.000000
2811986  2016-01-03   998666182   998666200    53.769095
2811987  2016-01-03   998666182   999457666   861.833614
2811988  2016-01-03   998666182   999999989   571.837564
2811989  2016-01-03   998666182   999999999   718.029125
2811990  2016-01-03   998666182  1040354684  1543.167599
2811991  2016-01-03   998666200   998666200     0.000000
2811992  2016-01-03   998666200   999457666   914.735117
2811993  2016-01-03   998666200   999999989   525.499906
2811994  2016-01-03   998666200   999999999   741.900354
2811995  2016-01-03   998666200  1040354684  1595.201739
2811996  2016-01-03   999457666   999457666     0.000000
2811997  2016-01-03   999457666   999999989  1415.650367
2811998  2016-01-03   999457666   999999999   976.858667
2811999  2016-01-03   999457666  1040354684   845.801865
2812000  2016-01-03   999999989   999999989     0.000000
2812001  2016-01-03   999999989   999999999   879.082007
2812002  2016-01-03   999999989  1040354684  1992.820765
2812003  2016-01-03   999999999   999999999     0.000000
2812004  2016-01-03   999999999  1040354684  1215.567459
2812005  2016-01-03  1040354684  1040354684     0.000000

[2812006 rows x 4 columns]